In [1]:
#API key - eq63r7svaqsta7wm_6bzrykdx2zj4jfnh
from san import *
import san
from decimal import Decimal
import pandas as pd
import datetime

In [2]:
#Getting all slugs/tickers/names/marketsegment/supply and dropping all with missing or 0 supply
data = san.get("projects/all")
data = data.dropna()
data['totalSupply'] = data['totalSupply'].astype(float)
data = data[data['totalSupply'] != 0]

In [3]:
#limiting only to cryptocurrency
crypto = data[data["marketSegment"] == "Cryptocurrency"]
crypto = crypto.sort_values(by = ["totalSupply"], ascending = False)
crypto

,marketSegment,name,slug,ticker,totalSupply
1748,Cryptocurrency,Dimecoin,dimecoin,DIME,5.559981e+11
1181,Cryptocurrency,Mooncoin,mooncoin,MOON,2.274909e+11
1854,Cryptocurrency,NewYorkCoin,newyorkcoin,NYC,1.437611e+11
1707,Cryptocurrency,Dogecoin,dogecoin,DOGE,1.291275e+11
1769,Cryptocurrency,eXPerience Chain,experience-chain,XPC,1.132155e+11
...,...,...,...,...,...
818,Cryptocurrency,Novacoin,novacoin,NVC,2.335757e+06
1818,Cryptocurrency,Bitcoin Rhodium,bitcoin-rhodium,XRC,1.316162e+06
1442,Cryptocurrency,ParallelCoin,parallelcoin,DUO,3.147186e+05
347,Cryptocurrency,Unobtanium,unobtanium,UNO,2.033284e+05


In [7]:
#Daily price list definition
#Definition will create daily return as percentage
def pricelist(slug):
    fullname = "price_usd/" + slug
    prices = san.get(
    fullname,
    from_date="2018-04-01",
    to_date="2021-04-01",
    interval="1h"
    )
    newname = slug+"_price"
    finaldf = prices.rename(columns={"value": newname})#For some reason it won't let me rename
    finaldf.index.name = 'datetime'
    finaldf.reset_index(inplace=True)
    finaldf['lagprice'] = finaldf[newname].shift(1)
    dr_slug = "dret_"+slug
    finaldf[dr_slug] = (finaldf[newname] - finaldf['lagprice'])/finaldf['lagprice']
    finaldf = finaldf.drop(['lagprice'], axis = 1)
    #note - right now I only care about daily return take this out if you want price data
    finaldf = finaldf.drop([newname], axis=1)
    return(finaldf)

In [8]:
#Running through all cryptocurrencies and adding price and daily returns
count = 0
allpdret = pd.DataFrame()
for i in crypto['slug'].tolist():
    holddf = pricelist(i)
    if count == 0:
        allpdret = holddf
    else:
        allpdret = pd.merge(allpdret, holddf, how = "left", on = "datetime")
    count = count+1

In [9]:
#Creating daily equal weighted market return (not really interested in value weight right now)

#will be used to calculate sum
temphold = allpdret[:]
temphold = temphold.fillna(0)
temphold = temphold.drop(['datetime'], axis = 1)
temphold['sumreturn'] = temphold.sum(axis = 1)
stuff1 = temphold['sumreturn'].tolist()

#will hold count of firms per year that have non-na returns
temp2hold = allpdret[:]
temp2hold = temp2hold.drop(['datetime'], axis = 1)
temp2hold['count'] = temp2hold.count(axis=1)
stuff2 = temp2hold['count'].tolist()

#will be used to calculate daily return
temp3hold = allpdret[:]
temp3hold['sumreturn'] = stuff1
temp3hold['count'] = stuff2
temp3hold['ew_avgret'] = temp3hold['sumreturn']/temp3hold['count']
ew_avgret = temp3hold['ew_avgret'].tolist()

#Can add value weighted with market cap

C:\Users\topcue.lee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\topcue.lee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\topcue.lee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [10]:
#Bitcoin return only - can create definition on slug to find every one if necessary
slug = "bitcoin"
dretname = "dret_" + slug
slug_data = allpdret[["datetime", dretname]]
slug_data['ew_market'] = ew_avgret
slug_data['excess_return'] = slug_data[dretname] - slug_data['ew_market']

C:\Users\topcue.lee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\topcue.lee\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
slug_data

,datetime,dret_bitcoin,ew_market,excess_return
0,2018-04-01 00:00:00+00:00,NaN,NaN,NaN
1,2018-04-01 01:00:00+00:00,0.003458,0.013251,-0.009794
2,2018-04-01 02:00:00+00:00,-0.011278,-0.014678,0.003400
3,2018-04-01 03:00:00+00:00,0.001080,0.004054,-0.002973
4,2018-04-01 04:00:00+00:00,0.000179,-0.007639,0.007818
...,...,...,...,...
26293,2021-04-01 19:00:00+00:00,0.003819,0.015432,-0.011613
26294,2021-04-01 20:00:00+00:00,-0.000840,-0.001665,0.000825
26295,2021-04-01 21:00:00+00:00,-0.001504,0.006875,-0.008380
26296,2021-04-01 22:00:00+00:00,0.003507,0.000947,0.002560
